In [1]:
import os
from dotenv import load_dotenv
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder

# 상위 경로의 open_api_key를 가져오기 위한 코드
env_path = os.path.join('..', '.env')
load_dotenv(dotenv_path=env_path)
openai_api_key = os.getenv('OPENAI_API_KEY')

# llm model 객체 생성
llm = ChatOpenAI(temperature=0.1)

# momory 객체 생성
memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    # memory_key="chat_history", default memory_key가 history 이기 때문에 이건 제외하고 이후 코드에서 history를 변수로 사용
    return_messages=True,
)

# prompt 생성
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful AI talking to a human"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

'''================================================================'''
''' LLMChain을 통한 chain 실습
-> off the shelf chain (자동으로 LLM으로부터 응답값을 가져오고 memory 저장 - 하지만 MessagesPlaceholder 를 프롬프트에 추가 필요)'''
# chain = LLMChain(
#     llm=llm,
#     memory=memory,
#     prompt=prompt,
#     verbose=True,
# )
# chain.predict(question="My name is Hyunjoong")

'''================================================================'''
'''Custom chain 실습'''

def load_memory(_):
    print(f"load_memory 함수 input: {_}")
    return memory.load_memory_variables({})["history"]

def invoke_chain(question):
    result = chain.invoke({"question": question})
    memory.save_context(
        {"input": question},
        {"output": result.content},
    )
    print(result)
    
chain = RunnablePassthrough.assign(history=load_memory) | prompt | llm


In [2]:
invoke_chain("Hi, My name is Hanni")

load_memory 함수 input: {'question': 'Hi, My name is Hanni'}
content='Hello Hanni! How can I assist you today?'


In [3]:
invoke_chain("Do you have any suggestions for dinner?")

load_memory 함수 input: {'question': 'Do you have any suggestions for dinner?'}
content='Sure! What type of cuisine are you in the mood for?'


In [4]:
invoke_chain("korean food or Italian food?")

load_memory 함수 input: {'question': 'korean food or Italian food?'}
content='Both Korean and Italian cuisines offer delicious options. For Korean food, you could try making bibimbap, bulgogi, or kimchi fried rice. For Italian food, you could make spaghetti carbonara, chicken parmesan, or a classic margherita pizza. Which one sounds more appealing to you?'


In [5]:
invoke_chain("what is my name?")

load_memory 함수 input: {'question': 'what is my name?'}
content='Your name is Hanni. How can I assist you further today, Hanni?'


In [6]:
memory.load_memory_variables({})

{'history': [SystemMessage(content='Hanni introduces themselves to the AI. The AI greets Hanni and asks how it can assist them today, suggesting dinner options based on the type of cuisine Hanni is in the mood for.'),
  HumanMessage(content='korean food or Italian food?'),
  AIMessage(content='Both Korean and Italian cuisines offer delicious options. For Korean food, you could try making bibimbap, bulgogi, or kimchi fried rice. For Italian food, you could make spaghetti carbonara, chicken parmesan, or a classic margherita pizza. Which one sounds more appealing to you?'),
  HumanMessage(content='what is my name?'),
  AIMessage(content='Your name is Hanni. How can I assist you further today, Hanni?')]}